In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

df_nhl_init = pd.read_csv('2021-2022_player_stats_naturalstattrick.csv')
df_ahl_init = pd.read_csv('2021-2022_AHL_all_players.csv')
df_name_update = pd.read_csv('2021-2022_name_updates.csv')

df_goals = pd.read_csv('2021-2022_dobber_goals.csv')
df_fg_init = pd.read_csv('2021-2022_dobber_fights.csv')
df_st_init = pd.read_csv('2021-2022_NHLcom_Player_Career.csv',encoding='latin1')
df_ass_init = pd.read_csv('2021-2022_dobber_assists.csv',encoding='latin1')
df_ca = pd.read_csv('2021-2022-EH_RAPM-EV.csv',encoding='latin1')
df_oz_init = pd.read_csv('2021-2022_NHLcom_Puck_OZ.csv',encoding='latin1')
df_pk_init = pd.read_csv('2021-2022_NHLcom_PK.csv',encoding='utf-8-sig')
df_ps_init = pd.read_csv('2021-2022_NHLcom_Shootouts.csv',encoding='utf-8-sig')
df_ex_init = pd.read_csv('2021-2022_NHLcom_Player_Career.csv',encoding='utf-8 sig')
df_ld_init = pd.read_csv('2021-2022-Elite_Prospects-Captaincy.csv', encoding = 'latin1')
df_pres = pd.read_csv('2021-2022_Sznajder_pressures.csv')
df_def = pd.read_csv('2021-2022_Sznajder_entry_defence.csv')
df_pos_ex = pd.read_csv('2021-2022_Sznajder_possession_exits.csv')
df_pos_ent = pd.read_csv('2021-2022_Sznajder_possession_carries.csv')

# VHL Player Ratings Model

Global Variables

In [ ]:
games_in_season = 82
min_rate = 54
max_rate = 99
max_rate2 = 89
max_rate_f_df = 83
min_rate_d_df = 65
rate_range = max_rate - min_rate
rate_range2 = max_rate2 = min_rate
rate_range_f_df = max_rate_f_df - min_rate
rate_range_d_df = max_rate - min_rate_d_df

#### Clean datasets so players have the same names in every df

In [ ]:
nhl = df_name_update['NHL Name'].to_list()
ahl = df_name_update['AHL Name'].to_list()
evolve = df_name_update['Evolving Hockey Name'].to_list()

df_ahl_init['Player'] = df_ahl_init['Player'].replace(ahl,nhl)
df_ca['Player'] = df_ca['Player'].replace(evolve,nhl)

### Players with more games in NHL than AHL

In [ ]:
df_ahl_init_cl = df_ahl_init.loc[:,['Player','GP']]
df_nhl_init_cl = df_nhl_init.loc[:,['Player','GP']]

df_ahl_init_cl['League'] = 'AHL'
df_nhl_init_cl['League'] = 'NHL'

df_names = pd.concat([df_nhl_init_cl,df_ahl_init_cl]).reset_index(drop=True)

df_names = df_names.iloc[df_names.groupby('Player')['GP'].idxmax()]
df_names['Player'] = df_names['Player'].str.lower()

df_nhl_names = df_names[df_names['League'] == 'NHL'].values.tolist()

names_list = [name[0] for name in df_nhl_names]
names_list = [name.lower() for name in names_list]
df_nhl_init['Player'] = df_nhl_init['Player'].str.lower()

file_clean = df_nhl_init[df_nhl_init['Player'].isin(names_list)]

file_clean

## Games Played Multiplier

In [ ]:
df_gm_mult = file_clean.loc[:,['Player','GP']]
df_gm_mult['add'] = np.where(((df_gm_mult['GP'] <= (games_in_season * 0.5)) & (df_gm_mult['GP'] >= (games_in_season * 0.35))),-1,0)
df_gm_mult['add'] = np.where(((df_gm_mult['GP'] < (games_in_season * 0.35)) & (df_gm_mult['GP'] >= (games_in_season * 0.2))),-2,df_gm_mult['add'])
df_gm_mult['add'] = np.where(((df_gm_mult['GP'] < (games_in_season * 0.2)) & (df_gm_mult['GP'] >= (games_in_season * 0.1))),-3,df_gm_mult['add'])
df_gm_mult['add'] = np.where((df_gm_mult['GP'] < (games_in_season * 0.1)),-4,df_gm_mult['add'])

df_gm_mult

# Scoring Rating (SC)

1. (EVG+SHG)/GP (70%)
2. PPG/GP (30%)

- Clean dataframe and only return pertinent columns
- Calculate Goals/GP, Points/GP and ixG/GP

In [ ]:
#Clean goals file for only NHL players
df_goals['Name'] = df_goals['Name'].str.lower()
df_goals2 = df_goals[df_goals['Name'].isin(names_list)]
df_goals2

In [ ]:
df_goals2 = df_goals2.loc[:,['Name','Pos','GP','EVG','PPG','SHG','G']]
df_goals2['EVGSHG'] = df_goals2['EVG'] + df_goals['SHG']
df_goals2['EVG/GP'] = df_goals2['EVGSHG']/df_goals2['GP']
df_goals2['PPG/GP'] = df_goals2['PPG']/df_goals2['GP']

Segment players who have played more than and less than 5 games

In [ ]:
df_goalsabove5 = df_goals2.loc[df_goals2['GP'] > 5]
df_goalsbelow5 = df_goals2.loc[df_goals2['GP'] <= 5]

Calculate EVG/GP, PPG/GP, SHG/GP z-score, z-score min, max, range and increment for players in > 5 games bucket

In [ ]:
df_3 = df_goalsabove5.loc[:,['Name','Pos','GP','EVG','PPG','SHG','G','EVG/GP','PPG/GP']]
df_3['even_zscore'] = zscore(df_3['EVG/GP'])
df_3['power_zscore'] = zscore(df_3['PPG/GP'])

z_max_even = df_3['even_zscore'].max()
z_min_even = df_3['even_zscore'].min()
z_max_power = df_3['power_zscore'].max()
z_min_power = df_3['power_zscore'].min()

z_range_even = z_max_even-z_min_even
z_range_power = z_max_power-z_min_power

z_incr_even = z_range_even/rate_range
z_incr_power = z_range_power/rate_range

## Goals/GP

Calculate Rating based on Goals/GP z-score for players in > 5 game bucket

In [ ]:
df_4 = df_3.loc[:,['Name','Pos','GP','EVG','PPG','SHG','G','even_zscore','power_zscore','EVG/GP','PPG/GP']]

df_4['EVG_Rate'] = 102-((z_max_even-df_4['even_zscore'])/z_incr_even)
df_4['PPG_Rate'] = 102-((z_max_power-df_4['power_zscore'])/z_incr_power)

df_4['SC'] = (df_4['EVG_Rate'] * 0.7) + (df_4['PPG_Rate'] * 0.3)

Calculate Rating for players in <- 5 games bucket by adding +1 rating to base 57 rating for every goal scored

In [ ]:
df_5 = df_goalsbelow5.loc[:,['Name','G']]
df_5['SC'] = 57+df_5['G']
df_goals_per_game = pd.concat([df_4,df_5], ignore_index=True)
df_goals_per_game.rename(columns={'Name':'Player'},inplace=True)
df_goals_per_game['Player'] = df_goals_per_game['Player'].str.lower()

df_sc_rate = df_goals_per_game.merge(df_gm_mult, how='left', on='Player')
df_sc_rate['SC'] = df_sc_rate['SC'] + df_sc_rate['add']
df_sc_rate = df_sc_rate.loc[:,['Player','SC']]

df_sc_rate.sort_values(by='SC',ascending=False, inplace=True)
df_sc_rate = df_sc_rate.reset_index(drop=True)
df_sc_rate.head(15)

# Checking (CK)

1. Hits/GP (60%)
2. Defender = Entry Defense Game Score (40%)
2. Forward = Forecheck Pressures/60 (40%)

In [ ]:
df_ck = file_clean.loc[:,['Player','Position','GP','TOI','Hits']]
df_ck['Hits/GP'] = df_ck['Hits']/df_ck['GP']

Segment players who have played more than and less than 50min

In [ ]:
df_ck_above5 = df_ck.loc[df_ck['GP'] > 5]
df_ck_below5 = df_ck.loc[df_ck['GP'] <= 5]

In [ ]:
df_ck2 = df_ck_above5.loc[:,['Player','Position','GP','TOI','Hits','Hits/GP']]
df_ck3 = df_ck_below5.loc[:,['Player','Position','GP','TOI','Hits','Hits/GP']]
df_ck2['hits_zscore'] = zscore(df_ck2['Hits/GP'])
df_ck3['hits_zscore'] = zscore(df_ck3['Hits/GP'])
z_max_hits = df_ck2['hits_zscore'].max()
z_min_hits = df_ck2['hits_zscore'].min()
z_max_hits2 = df_ck3['hits_zscore'].max()
z_min_hits2 = df_ck3['hits_zscore'].min()
z_range_hits = z_max_hits-z_min_hits
z_range_hits2 = z_max_hits2-z_min_hits2
z_incr_hits = z_range_hits/rate_range
z_incr_hits2 = z_range_hits2/(rate_range-20)

In [ ]:
df_ck2['Rating1'] = 100-((z_max_hits-df_ck2['hits_zscore'])/z_incr_hits)
df_ck3['Rating1'] = 79-((z_max_hits2-df_ck3['hits_zscore'])/z_incr_hits2)
df_ck4 = pd.concat([df_ck2,df_ck3])

Calculate Forwards Forecheck Recoveries/60 

In [ ]:
df_pres['Name'] = df_pres['Name'].str.lower().str.replace(u'\xa0', u' ')
df_pres = df_pres[df_pres['Name'].isin(names_list)]
df_pres

In [ ]:
df_pres_above40 = df_pres.loc[df_pres['TOI'] > 40]
df_pres_below40 = df_pres.loc[df_pres['TOI'] <= 40]

In [ ]:
df_pres_above40['pres_zscore'] = zscore(df_pres_above40['Pressures/60'])
df_pres_below40['pres_zscore'] = zscore(df_pres_below40['Pressures/60'])
z_max_pres = df_pres_above40['pres_zscore'].max()
z_min_pres = df_pres_above40['pres_zscore'].min()
z_max_pres2 = df_pres_below40['pres_zscore'].max()
z_min_pres2 = df_pres_below40['pres_zscore'].min()
z_range_pres = z_max_pres-z_min_pres
z_range_pres2 = z_max_pres2-z_min_pres2
z_incr_pres = z_range_pres/rate_range
z_incr_pres2 = z_range_pres2/(rate_range-20)

In [ ]:
df_pres_above40['Rating2'] = 102-((z_max_pres-df_pres_above40['pres_zscore'])/z_incr_pres)
df_pres_below40['Rating2'] = 79-((z_max_pres2-df_pres_below40['pres_zscore'])/z_incr_pres2)
df_pres_final = pd.concat([df_pres_above40,df_pres_below40])
df_pres_final.rename(columns={'Name' : 'Player'}, inplace=True)

display(df_pres_final.sort_values(by='Rating2',ascending=False))

Calculate Defenders Entry Defense Game Score

In [ ]:
df_def['Player'] = df_def['Player'].str.lower().str.replace(u'\xa0', u' ')
df_def_pos = df_def.merge(file_clean, on='Player', how='left')
df_def_pos = df_def_pos[df_def_pos['Player'].isin(names_list)]
df_def_pos = df_def_pos[df_def_pos['Position'] == 'D']
df_def_pos = df_def_pos[['Player','Defense','TOI']]
df_def_pos['Defense'] = np.where(df_def_pos['Defense'] >= 3, 3, df_def_pos['Defense'])
df_def_pos

In [ ]:
df_def_above40 = df_def_pos.loc[df_def_pos['TOI'] > 40]
df_def_below40 = df_def_pos.loc[df_def_pos['TOI'] <= 40]

In [ ]:
df_def_above40['def_zscore'] = zscore(df_def_above40['Defense'])
df_def_below40['def_zscore'] = zscore(df_def_below40['Defense'])
z_max_def = df_def_above40['def_zscore'].max()
z_min_def = df_def_above40['def_zscore'].min()
z_max_def2 = df_def_below40['def_zscore'].max()
z_min_def2 = df_def_below40['def_zscore'].min()
z_range_def = z_max_def-z_min_def
z_range_def2 = z_max_def2-z_min_def2
z_incr_def = z_range_def/rate_range
z_incr_def2 = z_range_def2/(rate_range-20)

In [ ]:
df_def_above40['Rating2'] = 102-((z_max_def-df_def_above40['def_zscore'])/z_incr_def)
df_def_below40['Rating2'] = 79-((z_max_def2-df_def_below40['def_zscore'])/z_incr_def2)
df_def_final = pd.concat([df_def_above40,df_def_below40])

display(df_def_final.sort_values(by='Rating2',ascending=False))

In [ ]:
df_ck_concat = pd.concat([df_pres_final,df_def_final])

df_ck_merge = df_ck4.merge(df_ck_concat, how='left', on='Player')

df_ck_merge.drop_duplicates(inplace=True)
df_ck_merge['CK'] = (df_ck_merge['Rating1'] * 0.7) + (df_ck_merge['Rating2'] * 0.3)
df_ck_merge['CK'] = df_ck_merge['CK'].fillna(57)

df_ck_rate = df_ck_merge[['Player','Position','CK']]

df_ck_rate.sort_values(by='CK', ascending=False, inplace=True)

df_ck_rate.head(15).reset_index(drop=True)

# Fighting (FG)

1. Fights/GP (100%)

In [ ]:
df_fg_init['Name'] = df_fg_init['Name'].str.lower()
df_fg = df_fg_init[df_fg_init['Name'].isin(names_list)]

df_fg

Segment players into those who have played more or less than 5 games

In [ ]:
df_fg_above5 = df_fg.loc[df_fg['GP'] > 5]
df_fg_below5 = df_fg.loc[df_fg['GP'] <= 5]

In [ ]:
df_fg2 = df_fg_above5.loc[:,['Name','GP','Fights']]
df_fg2['Fights/GP'] = df_fg2['Fights']/df_fg2['GP']
df_fg2['fights_zscore'] = zscore(df_fg2['Fights/GP'])
z_max_fights = df_fg2['fights_zscore'].max()
z_min_fights = df_fg2['fights_zscore'].min()
z_range_fights = z_max_fights-z_min_fights
z_incr_fights = z_range_fights/(rate_range-10)
df_fg2['FG'] = 99-((z_max_fights-df_fg2['fights_zscore'])/z_incr_fights)
display(df_fg2.sort_values(by=['FG'],ascending=False))

In [ ]:
df_fg3 = df_fg_below5.loc[:,['Name','GP','Fights']]
df_fg3['FG'] = 55 + df_fg3['Fights']

In [ ]:
df_fg_concat = df_fg2.append(df_fg3)

df_fg_rate = df_fg_concat[['Name','FG']]
df_fg_rate['Name'] = df_fg_rate['Name'].str.lower()
df_fg_rate.rename(columns={'Name' : 'Player'},inplace=True)

df_fg_rate.sort_values(by='FG',ascending=False, inplace=True)
df_fg_rate = df_fg_rate.reset_index(drop=True)
df_fg_rate.head(15)

# Discipline (DI)

1. Minor Penalties Taken (85%)
2. Penalties Drawn / Total Penalties Taken (15%)

### Minor Penalties Taken/GP

In [ ]:
df_di = file_clean.loc[:,['Player','GP','TOI','Minor','Total Penalties','Penalties Drawn']]
df_di['Penalties/GP'] = df_di['Minor']/df_di['GP']
df_di['Drawn/Taken'] = df_di['Penalties Drawn']/df_di['Total Penalties']
df_di['Drawn/Taken'] = df_di['Drawn/Taken'].fillna(0).replace([np.inf],3)

Segment players who have played more than and less than 100min

In [ ]:
df_di_above125 = df_di.loc[df_di['GP'] > 10]
df_di_below125 = df_di.loc[df_di['GP'] <= 10]

In [ ]:
df_di2 = df_di_above125.loc[:,['Player','GP','TOI','Penalties/GP','Drawn/Taken']]
df_di2['penalties_zscore'] = zscore(df_di2['Penalties/GP'])
df_di2['takendrawn_zscore'] = zscore(df_di2['Drawn/Taken'])

In [ ]:
z_max_penalties = df_di2['penalties_zscore'].max()
z_min_penalties = df_di2['penalties_zscore'].min()
z_max_takendrawn = df_di2['takendrawn_zscore'].max()
z_min_takendrawn = df_di2['takendrawn_zscore'].min()
z_range_penalties = z_max_penalties-z_min_penalties
z_range_takendrawn = z_max_takendrawn-z_min_takendrawn
z_incr_penalties = z_range_penalties/rate_range
z_incr_takendrawn = z_range_takendrawn/rate_range

In [ ]:
df_di2['Rating1'] = 57+((z_max_penalties-df_di2['penalties_zscore'])/z_incr_penalties)
df_di2['Rating2'] = 99-((z_max_takendrawn-df_di2['takendrawn_zscore'])/z_incr_takendrawn)
df_di2['DI'] = (df_di2['Rating1'] * 0.85) + (df_di2['Rating2'] * 0.15)
display(df_di2.sort_values(by=['DI'],ascending=False))

In [ ]:
df_di3 = df_di_below125.loc[:,['Player','GP','TOI','Penalties/GP','Drawn/Taken']]
df_di3['penalties_zscore'] = zscore(df_di3['Penalties/GP'])
df_di3['takendrawn_zscore'] = zscore(df_di3['Drawn/Taken'])

In [ ]:
z_max_penalties2 = df_di3['penalties_zscore'].max()
z_min_penalties2 = df_di3['penalties_zscore'].min()
z_max_takendrawn2 = df_di3['takendrawn_zscore'].max()
z_min_takendrawn2 = df_di3['takendrawn_zscore'].min()
z_range_penalties2 = z_max_penalties2-z_min_penalties2
z_range_takendrawn2 = z_max_takendrawn2-z_min_takendrawn2
z_incr_penalties2 = z_range_penalties2/rate_range
z_incr_takendrawn2 = z_range_takendrawn2/rate_range

In [ ]:
df_di3['Rating1'] = 59+((z_max_penalties2-df_di3['penalties_zscore'])/z_incr_penalties2)
df_di3['Rating2'] = 99-((z_max_takendrawn2-df_di3['takendrawn_zscore'])/z_incr_takendrawn2)
df_di3['DI'] = (df_di3['Rating1'] * 0.9) + (df_di3['Rating2'] * 0.1)
df_di3['DI'] = df_di3['DI'] - 15
display(df_di3.sort_values(by=['DI'],ascending=False).head(250))

In [ ]:
df_di_concat = df_di2.append(df_di3)
df_di_rate = df_di_concat[['Player','DI']]

df_di_rate['Player'] = df_di_rate['Player'].str.lower()

df_di_rate.sort_values(by='DI',ascending=False, inplace=True)

df_di_rate.head(15)

# Strength (ST)

1. Weight/Height Ratio (100%)

In [ ]:
df_st_init['Player'] = df_st_init['Player'].str.lower()
df_st = df_st_init[df_st_init['Player'].isin(names_list)]

In [ ]:
df_st2 = df_st.loc[:,['Player','Height','Weight']]

Convert Height to cm

In [ ]:
def parse_ht(ht):
    ht_ = ht.split("' ")
    ft_ = float(ht_[0])
    in_ = float(ht_[1].replace("\"",""))
    return round((12*ft_) + in_)

df_st2['height_in'] = df_st2['Height'].apply(lambda x:parse_ht(x))

In [ ]:
df_st2['ratio_zscore'] = zscore(df_st2['Weight']/df_st2['height_in'])
z_max_st = df_st2['ratio_zscore'].max()
z_min_st = df_st2['ratio_zscore'].min()
z_range_st = z_max_st-z_min_st
z_incr_st = z_range_st/rate_range
df_st2['ST'] = 99-((z_max_st-df_st2['ratio_zscore'])/z_incr_st)
df_st_rate = df_st2[['Player','height_in','ST']]

df_st_rate['Player'] = df_st_rate['Player'].str.lower()

df_st_rate.sort_values(by='ST',ascending=False,inplace=True)

df_st_rate.head(15).reset_index(drop=True)

# Endurance (EN)

1. Average TOI (100%)

In [ ]:
df_en = file_clean.loc[:,['Player','GP','TOI']]
df_en['ATOI'] = df_en['TOI']/df_en['GP']

In [ ]:
df_en_above125 = df_en.loc[df_en['TOI'] > 125]
df_en_below125 = df_en.loc[df_en['TOI'] <= 125]

In [ ]:
df_en2 = df_en_above125.loc[:,['Player','GP','TOI','ATOI']]
df_en2['ATOI'] = np.where(df_en2['ATOI'] < 8, 8, df_en2['ATOI'])
df_en2['atoi_zscore'] = zscore(df_en2['ATOI'])
z_max_en = df_en2['atoi_zscore'].max()
z_min_en = df_en2['atoi_zscore'].min()
z_range_en = z_max_en-z_min_en
z_incr_en = z_range_en/rate_range
df_en2['EN'] = 99-((z_max_en-df_en2['atoi_zscore'])/z_incr_en)
display(df_en2.sort_values(by=['EN'],ascending=False))

In [ ]:
df_en3 = df_en_below125.loc[:,['Player','GP','TOI','ATOI']]
df_en3['EN'] = 57 + df_en3['ATOI']
display(df_en3.sort_values(by=['EN'],ascending=False))

In [ ]:
df_en_concat = df_en2.append(df_en3)
df_en_rate = df_en_concat[['Player','EN']]

df_en_rate['Player'] = df_en_rate['Player'].str.lower()

df_en_rate = df_en_rate.sort_values(by=['EN'],ascending=False)
df_en_rate.head(15).reset_index(drop=True)

# Durability (DU)

1. Games Played last 3 seasons vs. 82 game season (Last Season = 60%, 2 Seasons Ago = 30%, 3 Seasons Ago = 10%) (90%)
2. Hits Given + Hits Taken (10%)

In [ ]:
df_dudsh2 = pd.read_csv('2020-2021_player_stats_naturalstattrick.csv')
df_dudsh3 = pd.read_csv('2019-2020_player_stats_naturalstattrick.csv')

In [ ]:
df_dudash1 = file_clean.loc[:,['Player','GP']]
df_dudash2 = df_dudsh2.loc[:,['Player','GP']]
df_dudash2['Player'] = df_dudash2['Player'].str.lower()
df_dudash3 = df_dudsh3.loc[:,['Player','GP']]
df_dudash3['Player'] = df_dudsh3['Player'].str.lower()

df_dudash1['GP%'] = df_dudash1['GP']/82
df_dudash2['GP%'] = df_dudash2['GP']/56
df_dudash3['GP%'] = df_dudash3['GP']/70

Calculate DU Rating for most recent season

In [ ]:
df_dudash1_above5 = df_dudash1.loc[df_dudash1['GP'] >= 20]
df_dudash1_below5 = df_dudash1.loc[df_dudash1['GP'] < 20]

In [ ]:
df_dudash1_above = df_dudash1_above5.loc[:,['Player','GP','GP%']]
df_dudash1_above['gp%_zscore'] = zscore(df_dudash1_above['GP%'])
z_max_dudash1 = df_dudash1_above['gp%_zscore'].max()
z_min_dudash1 = df_dudash1_above['gp%_zscore'].min()
z_range_dudash1 = z_max_dudash1-z_min_dudash1
z_incr_dudash1 = z_range_dudash1/rate_range
df_dudash1_above['DU_Dash1'] = 99-((z_max_dudash1-df_dudash1_above['gp%_zscore'])/z_incr_dudash1)
display(df_dudash1_above.sort_values(by=['DU_Dash1'],ascending=True))

In [ ]:
df_dudash1_below = df_dudash1_below5.loc[:,['Player','GP','GP%']]
df_dudash1_below['DU_Dash1'] = 55

In [ ]:
df_dudash1_concat = df_dudash1_above.append(df_dudash1_below).fillna(0)
display(df_dudash1_concat)

Calculate DU Rating for two seasons ago

In [ ]:
df_dudash2_above5 = df_dudash2.loc[df_dudash2['GP'] >= 10]
df_dudash2_below5 = df_dudash2.loc[df_dudash2['GP'] < 10]

In [ ]:
df_dudash2_above = df_dudash2_above5.loc[:,['Player','GP','GP%']]
df_dudash2_above['gp%_zscore'] = zscore(df_dudash2_above['GP%'])
z_max_dudash2 = df_dudash2_above['gp%_zscore'].max()
z_min_dudash2 = df_dudash2_above['gp%_zscore'].min()
z_range_dudash2 = z_max_dudash2-z_min_dudash2
z_incr_dudash2 = z_range_dudash2/rate_range
df_dudash2_above['DU_Dash2'] = 99-((z_max_dudash2-df_dudash2_above['gp%_zscore'])/z_incr_dudash2)
display(df_dudash2_above.sort_values(by=['DU_Dash2'],ascending=True))

In [ ]:
df_dudash2_below = df_dudash2_below5.loc[:,['Player','GP','GP%']]
df_dudash2_below['DU_Dash2'] = 55
display(df_dudash2_below)

In [ ]:
df_dudash2_concat = df_dudash2_above.append(df_dudash2_below).fillna(0)
display(df_dudash2_concat)

Calculate DU Rating for three seasons ago

In [ ]:
df_dudash3_above5 = df_dudash3.loc[df_dudash3['GP'] >= 10]
df_dudash3_below5 = df_dudash3.loc[df_dudash3['GP'] < 10]

In [ ]:
df_dudash3_above = df_dudash3_above5.loc[:,['Player','GP','GP%']]
df_dudash3_above['gp%_zscore'] = zscore(df_dudash3_above['GP%'])
z_max_dudash3 = df_dudash3_above['gp%_zscore'].max()
z_min_dudash3 = df_dudash3_above['gp%_zscore'].min()
z_range_dudash3 = z_max_dudash3-z_min_dudash3
z_incr_dudash3 = z_range_dudash3/rate_range
df_dudash3_above['DU_Dash3'] = 99-((z_max_dudash3-df_dudash3_above['gp%_zscore'])/z_incr_dudash3)
display(df_dudash3_above.sort_values(by=['DU_Dash3'],ascending=True))

In [ ]:
df_dudash3_below = df_dudash3_below5.loc[:,['Player','GP','GP%']]
df_dudash3_below['DU_Dash3'] = 55
display(df_dudash3_below)

In [ ]:
df_dudash3_concat = df_dudash3_above.append(df_dudash3_below).fillna(0)
display(df_dudash3_concat)

Merge all 3 season dataframes

In [ ]:
df_du_m = pd.merge(df_dudash1_concat,df_dudash2_concat, how='outer',on=['Player'])
df_du_merged = pd.merge(df_du_m,df_dudash3_concat, how='outer',on=['Player']).fillna(55)
display(df_du_merged)

In [ ]:
df_du_gp = df_du_merged.loc[:,['Player','GP_x','DU_Dash1','GP_y','DU_Dash2','GP','DU_Dash3']]
df_du_gp['DU_GP'] = (df_du_gp['DU_Dash1'] * 0.6) + (df_du_gp['DU_Dash2'] * 0.3) + (df_du_gp['DU_Dash3'] * 0.1)
display(df_du_gp.sort_values(by=['DU_GP'],ascending=False))

Sum up Hits Given and Hits Taken per game

In [ ]:
df_du_hits = file_clean.loc[:,['Player','GP','Hits','Hits Taken']]
df_du_hits['Hits_Sum'] = df_du_hits['Hits'] + df_du_hits['Hits Taken']
df_du_hits['Hits/GP']  = df_du_hits['Hits_Sum']/df_du_hits['GP']

In [ ]:
df_du_hits_above5 = df_du_hits.loc[df_du_hits['GP'] >= 5]
df_du_hits_below5 = df_du_hits.loc[df_du_hits['GP'] < 5]

In [ ]:
df_du_hits_above = df_du_hits_above5.loc[:,['Player','GP','Hits','Hits Taken','Hits_Sum','Hits/GP']]
df_du_hits_above['hits_zscore'] = zscore(df_du_hits_above['Hits/GP'])
z_max_du_hits = df_du_hits_above['hits_zscore'].max()
z_min_du_hits = df_du_hits_above['hits_zscore'].min()
z_range_du_hits = z_max_du_hits-z_min_du_hits
z_incr_du_hits = z_range_du_hits/rate_range
df_du_hits_above['Hits_Rating'] = 99-((z_max_du_hits-df_du_hits_above['hits_zscore'])/z_incr_du_hits)
display(df_du_hits_above.sort_values(by=['Hits_Rating'],ascending=False).head(25))

In [ ]:
df_du_hits_below = df_du_hits_below5.loc[:,['Player','GP','Hits','Hits Taken','Hits_Sum','Hits/GP']]
df_du_hits_below['hits_zscore'] = zscore(df_du_hits_below['Hits/GP'])
z_max_du_hits2 = df_du_hits_below['hits_zscore'].max()
z_min_du_hits2 = df_du_hits_below['hits_zscore'].min()
z_range_du_hits2 = z_max_du_hits2-z_min_du_hits2
z_incr_du_hits2 = z_range_du_hits2/(rate_range-10)
df_du_hits_below['Hits_Rating'] = 89-((z_max_du_hits2-df_du_hits_below['hits_zscore'])/z_incr_du_hits2)
display(df_du_hits_below.sort_values(by=['Hits_Rating'],ascending=False))

In [ ]:
df_du_hits_concat = df_du_hits_above.append(df_du_hits_below)
display(df_du_hits_concat)

In [ ]:
df_du_m_final = pd.merge(df_du_gp,df_du_hits_concat, how='left',on=['Player']).fillna(55)
df_du_final = df_du_m_final.loc[:,['Player','DU_GP','Hits_Rating']]
df_du_final['DU'] = (df_du_final['DU_GP'] * 0.9) + (df_du_final['Hits_Rating'] * 0.1)

df_du_rate = df_du_final[['Player','DU']]
df_du_rate = df_du_rate.groupby(['Player']).mean().reset_index()

df_du_rate['Player'] = df_du_rate['Player'].str.lower()

df_du_rate.sort_values(by='DU',ascending=False,inplace=True)
df_du_rate.head(15).reset_index(drop=True)

# Puck Handling (PH)

1. Points/GP (100%)

In [ ]:
df_ph = file_clean.loc[:,['Player','GP','TOI','Total Points','iHDCF']]
df_ph['Points/GP'] = df_ph['Total Points']/df_ph['GP']

In [ ]:
df_ph_above50 = df_ph.loc[df_ph['TOI'] > 200]
df_ph_below50 = df_ph.loc[df_ph['TOI'] <= 200]

In [ ]:
df_ph_above50['ppg_zscore'] = zscore(df_ph_above50['Points/GP'])
df_ph_below50['ppg_zscore'] = zscore(df_ph_below50['Points/GP'])

z_max_ph_above50 = df_ph_above50['ppg_zscore'].max()
z_min_ph_above50 = df_ph_above50['ppg_zscore'].min()
z_range_ph_above50 = z_max_ph_above50-z_min_ph_above50
z_incr_ph_above50 = z_range_ph_above50/rate_range

z_max_ph_below50 = df_ph_below50['ppg_zscore'].max()
z_min_ph_below50 = df_ph_below50['ppg_zscore'].min()
z_range_ph_below50 = z_max_ph_below50-z_min_ph_below50
z_incr_ph_below50 = z_range_ph_below50/(rate_range-29)

df_ph_above50['PH'] = 99-((z_max_ph_above50-df_ph_above50['ppg_zscore'])/z_incr_ph_above50)
df_ph_below50['PH'] = 70-((z_max_ph_below50-df_ph_below50['ppg_zscore'])/z_incr_ph_below50)

df_ph_below50

In [ ]:
df_ph_concat = pd.concat([df_ph_above50,df_ph_below50])
df_ph_concat = df_ph_concat[['Player','PH']]

df_ph_rate = df_ph_concat.merge(df_gm_mult,how='left',on='Player')
df_ph_rate['PH'] = df_ph_rate['PH'] + df_ph_rate['add']
df_ph_rate = df_ph_rate[['Player','PH']]

df_ph_rate.sort_values(by='PH',ascending=False,inplace=True)

df_ph_rate.head(15).reset_index(drop=True)

# Faceoffs (FO)

1. FOW% (100%)
2. Sliding scale for every 200 draws taken less than top draw taker, subtract 1 additional rating, from max to 800 draws

In [ ]:
df_fo = file_clean.loc[:,['Player','Position','GP','Faceoffs Won','Faceoffs Lost','Faceoffs %']]
df_fo = df_fo.replace('-',0)
df_fo['Faceoffs %'] = df_fo['Faceoffs %'].astype(float).astype(int)
df_fo['Faceoffs Taken'] = df_fo['Faceoffs Won'] + df_fo['Faceoffs Lost']
df_fo_above75 = df_fo.loc[df_fo['Faceoffs Taken'] >= 75]
df_fo_below75 = df_fo.loc[df_fo['Faceoffs Taken'] < 75]

In [ ]:
df_fo2 = df_fo_above75.loc[:,['Player','Position','GP','Faceoffs Won','Faceoffs Lost','Faceoffs Taken','Faceoffs %']]
df_fo2['fo_zscore'] = zscore(df_fo2['Faceoffs %'])
z_max_fo = df_fo2['fo_zscore'].max()
z_min_fo = df_fo2['fo_zscore'].min()
z_range_fo = z_max_fo-z_min_fo
z_incr_fo = z_range_fo/rate_range
df_fo2['FO'] = 99-((z_max_fo-df_fo2['fo_zscore'])/z_incr_fo)

In [ ]:
df_fo3 = df_fo_below75.loc[:,['Player','Position','GP','Faceoffs Won','Faceoffs Lost','Faceoffs Taken','Faceoffs %']]
df_fo3['FO'] = np.where(df_fo3['GP'] < 5, 50, 52 + (df_fo3['Faceoffs Won']*(df_fo3['Faceoffs %']/100)))
display(df_fo3.sort_values(by=['FO'],ascending=False))

1. If position is D, set FO rating to 40
2. Add sliding scale adjustment (O'Reilly has max of 1661 faceoffs so base off 1600)

In [ ]:
df_fo_concat = pd.concat([df_fo2,df_fo3])
display(df_fo_concat.sort_values(by=['Faceoffs Taken'],ascending=False))

In [ ]:
def rounding(x, base=200):
    return int(base * round(x/base))
df_fo_concat['FO'] = np.where(df_fo_concat['Faceoffs Taken'] > 75, df_fo_concat['FO'] - ((1600 - df_fo_concat['Faceoffs Taken'].apply(lambda x: rounding(x,base=200)))/200), df_fo_concat['FO'])
df_fo_concat['FO'] = np.where(df_fo_concat['Position'] == 'D', 40, df_fo_concat['FO'])
df_fo_concat['FO'] = np.where((df_fo_concat['Position'] != 'D') & (df_fo_concat['Faceoffs Taken'] < 2), 51, df_fo_concat['FO'])
df_fo_rate = df_fo_concat[['Player','FO']]

df_fo_rate['Player'] = df_fo_rate['Player'].str.lower()

df_fo_rate.sort_values(by='FO',ascending=False,inplace=True)

df_fo_rate.head(15).reset_index(drop=True)

# Passing (PA)

1. EV + SH Primary assist (40%)
2. PP Primary assist (30%)
4. EV + SH Secondary assist (25%)
5. PP Secondary assist (5%)

In [ ]:
df_ass_init['Name'] = df_ass_init['Name'].str.lower()
df_ass = df_ass_init[df_ass_init['Name'].isin(names_list)]

df_ass['Name'] = np.where('Â' in df_ass['Name'], df_ass['Name'].str[3:], df_ass['Name'])
df_ass['Name'] = df_ass['Name'].str.lower()

df_pa = df_ass.loc[:,['Name','GP','Prim.','Sec.','EV Prim.','PP Prim.','SH Prim.','EV Sec.','PP Sec.','SH Sec.']]
df_pa['EVSH1/GP'] = (df_pa['EV Prim.'] + df_pa['SH Prim.'])/df_pa['GP']
df_pa['PP1/GP'] = df_pa['PP Prim.']/df_pa['GP']
df_pa['EVSH2/GP'] = (df_pa['EV Sec.'] + df_pa['SH Sec.'])/df_pa['GP']
df_pa['PP2/GP'] = df_pa['PP Sec.']/df_pa['GP']
df_pa_above5 = df_pa.loc[df_pa['GP'] > 5]
df_pa_below5 = df_pa.loc[df_pa['GP'] <= 5]

In [ ]:
df_pa2 = df_pa_above5.loc[:,['Name','EVSH1/GP','PP1/GP','EVSH2/GP','PP2/GP']]
df_pa2['evsh1_zscore'] = zscore(df_pa2['EVSH1/GP'])
df_pa2['pp1_zscore'] = zscore(df_pa2['PP1/GP'])
df_pa2['evsh2_zscore'] = zscore(df_pa2['EVSH2/GP'])
df_pa2['pp2_zscore'] = zscore(df_pa2['PP2/GP'])

z_max_evsh1 = df_pa2['evsh1_zscore'].max()
z_min_evsh1 = df_pa2['evsh1_zscore'].min()
z_range_evsh1 = z_max_evsh1-z_min_evsh1
z_incr_evsh1 = z_range_evsh1/rate_range

z_max_pp1 = df_pa2['pp1_zscore'].max()
z_min_pp1 = df_pa2['pp1_zscore'].min()
z_range_pp1 = z_max_pp1-z_min_pp1
z_incr_pp1 = z_range_pp1/rate_range

z_max_evsh2 = df_pa2['evsh2_zscore'].max()
z_min_evsh2 = df_pa2['evsh2_zscore'].min()
z_range_evsh2 = z_max_evsh2-z_min_evsh2
z_incr_evsh2 = z_range_evsh2/rate_range

z_max_pp2 = df_pa2['pp2_zscore'].max()
z_min_pp2 = df_pa2['pp2_zscore'].min()
z_range_pp2 = z_max_pp2-z_min_pp2
z_incr_pp2 = z_range_pp2/rate_range

df_pa2['EVSH1_Rating'] = 99-((z_max_evsh1-df_pa2['evsh1_zscore'])/z_incr_evsh1)
df_pa2['PP1_Rating'] = 99-((z_max_pp1-df_pa2['pp1_zscore'])/z_incr_pp1)
df_pa2['EVSH2_Rating'] = 99-((z_max_evsh2-df_pa2['evsh2_zscore'])/z_incr_evsh2)
df_pa2['PP2_Rating'] = 99-((z_max_pp2-df_pa2['pp2_zscore'])/z_incr_pp2)

df_pa2['PA'] = (df_pa2['EVSH1_Rating'] * 0.40) + (df_pa2['PP1_Rating'] * 0.30) + (df_pa2['EVSH2_Rating'] * 0.25) + (df_pa2['PP2_Rating'] * 0.05)
df_pa2['PA'] = df_pa2['PA'] + (99 - df_pa2['PA'].max())
display(df_pa2.sort_values(by=['PA'],ascending=False))

In [ ]:
df_pa3 = df_pa_below5.loc[:,['Name','Prim.','Sec.']]
df_pa3['PA'] = 59 + (df_pa3['Prim.'] * 2) + (df_pa3['Sec.'] * 1)
display(df_pa3.sort_values(by=['PA'],ascending=False))                    

In [ ]:
df_pa_concat = pd.concat([df_pa2,df_pa3])
df_pa_concat.rename(columns={'Name': 'Player'}, inplace=True)
df_pa_calc = df_pa_concat.merge(df_gm_mult, how='left', on='Player')
df_pa_calc['PA'] = df_pa_calc['PA'] + df_pa_calc['add']
df_pa_rate = df_pa_concat[['Player','PA']]

df_pa_rate.sort_values(by='PA',ascending=False,inplace=True)

df_pa_rate.head(15).reset_index(drop=True)

# Defence (DF)

1. xGA/60 (50%)
2. PK TOI/GP (30%)
3. Defensive Zone Starts (20%)

## xGA/60

In [ ]:
df_ca.sort_values(by=['Player','GP'],ascending=False,inplace=True)
df_ca.drop_duplicates(subset='Player',keep='first',inplace=True)
df_ca2 = df_ca.loc[:,['Player','GP','Position','TOI','xGA/60']]
df_ca2['xGA/60'] = df_ca2['xGA/60'] * -1
df_ca2f = df_ca2.loc[df_ca2['Position'] != 'D']
df_ca2d = df_ca2.loc[df_ca2['Position'] == 'D']

z_max_caf = df_ca2f['xGA/60'].max()
z_min_caf = df_ca2f['xGA/60'].min()
z_range_caf = z_max_caf-z_min_caf
z_incr_caf = z_range_caf/rate_range_f_df

z_max_cad = df_ca2d['xGA/60'].max()
z_min_cad = df_ca2d['xGA/60'].min()
z_range_cad = z_max_cad-z_min_cad
z_incr_cad = z_range_cad/rate_range_d_df

df_ca2f['xGA60_Rating'] = 85-((z_max_caf-df_ca2f['xGA/60'])/z_incr_caf)
df_ca2d['xGA60_Rating'] = 100-((z_max_cad-df_ca2d['xGA/60'])/z_incr_cad)
df_ca_concat = pd.concat([df_ca2f,df_ca2d])
df_ca_concat['Player'] = df_ca_concat['Player'].str.lower()
display(df_ca_concat.sort_values(by=['xGA60_Rating'], ascending=False))

## Defensive Zone Starts

In [ ]:
df_oz_init['skaterFullName'] = df_oz_init['skaterFullName'].str.lower()
df_oz = df_oz_init[df_oz_init['skaterFullName'].isin(names_list)]

df_oz2 = df_oz.loc[:,['skaterFullName','positionCode','gamesPlayed','zoneStartPct']]
df_oz2['zoneStartPct'] = np.where(df_oz2['zoneStartPct'] < 0.2, 0.2, df_oz2['zoneStartPct'])
df_oz2['zoneStartPct'] = np.where(df_oz2['zoneStartPct'] > 0.75, 0.75, df_oz2['zoneStartPct']) 

df_oz2f = df_oz2.loc[df_oz2['positionCode'] != 'D']
df_oz2d = df_oz2.loc[df_oz2['positionCode'] == 'D']

df_oz2f_above10 = df_oz2f.loc[df_oz2f['gamesPlayed'] >= 10]
df_oz2f_below10 = df_oz2f.loc[df_oz2f['gamesPlayed'] < 10]
df_oz2d_above10 = df_oz2d.loc[df_oz2d['gamesPlayed'] >= 10]
df_oz2d_below10 = df_oz2d.loc[df_oz2d['gamesPlayed'] < 10]

df_oz2f_a10 = df_oz2f_above10.loc[:,['skaterFullName','positionCode','gamesPlayed','zoneStartPct']]
df_oz2f_a10['zoneStartPct'] = 1 -df_oz2f_a10['zoneStartPct']
df_oz2f_a10['oz_zscore'] = zscore(df_oz2f_a10['zoneStartPct'])
z_max_oz2f_a10 = df_oz2f_a10['oz_zscore'].max()
z_min_oz2f_a10 = df_oz2f_a10['oz_zscore'].min()
z_range_oz2f_a10 = z_max_oz2f_a10-z_min_oz2f_a10
z_incr_oz2f_a10 = z_range_oz2f_a10/rate_range_f_df

df_oz2f_b10 = df_oz2f_below10.loc[:,['skaterFullName','positionCode','gamesPlayed','zoneStartPct']]
df_oz2f_b10['zoneStartPct'] = 1 - df_oz2f_b10['zoneStartPct'].fillna(0)
df_oz2f_b10['oz_zscore'] = zscore(df_oz2f_b10['zoneStartPct'])
z_max_oz2f_b10 = df_oz2f_b10['oz_zscore'].max()
z_min_oz2f_b10 = df_oz2f_b10['oz_zscore'].min()
z_range_oz2f_b10 = z_max_oz2f_b10-z_min_oz2f_b10
z_incr_oz2f_b10 = z_range_oz2f_b10/(rate_range_f_df-10)

df_oz2d_a10 = df_oz2d_above10.loc[:,['skaterFullName','positionCode','gamesPlayed','zoneStartPct']]
df_oz2d_a10['zoneStartPct'] = 1 -df_oz2d_a10['zoneStartPct']
df_oz2d_a10['oz_zscore'] = zscore(df_oz2d_a10['zoneStartPct'])
z_max_oz2d_a10 = df_oz2d_a10['oz_zscore'].max()
z_min_oz2d_a10 = df_oz2d_a10['oz_zscore'].min()
z_range_oz2d_a10 = z_max_oz2d_a10-z_min_oz2d_a10
z_incr_oz2d_a10 = z_range_oz2d_a10/rate_range_d_df

df_oz2d_b10 = df_oz2d_below10.loc[:,['skaterFullName','positionCode','gamesPlayed','zoneStartPct']]
df_oz2d_b10['zoneStartPct'] = 1 - df_oz2d_b10['zoneStartPct']
df_oz2d_b10['zoneStartPct'].fillna(0.25,inplace=True)
df_oz2d_b10['oz_zscore'] = zscore(df_oz2d_b10['zoneStartPct'])
z_max_oz2d_b10 = df_oz2d_b10['oz_zscore'].max()
z_min_oz2d_b10 = df_oz2d_b10['oz_zscore'].min()
z_range_oz2d_b10 = z_max_oz2d_b10-z_min_oz2d_b10
z_incr_oz2d_b10 = z_range_oz2d_b10/(rate_range_d_df-10)

df_oz2d_a10['OZ%_Rating'] = 100-((z_max_oz2d_a10-df_oz2d_a10['oz_zscore'])/z_incr_oz2d_a10)
df_oz2d_b10['OZ%_Rating'] = 80-((z_max_oz2d_b10-df_oz2d_b10['oz_zscore'])/z_incr_oz2d_b10)
df_oz2f_a10['OZ%_Rating'] = 83-((z_max_oz2f_a10-df_oz2f_a10['oz_zscore'])/z_incr_oz2f_a10)
df_oz2f_b10['OZ%_Rating'] = 70-((z_max_oz2f_b10-df_oz2f_b10['oz_zscore'])/z_incr_oz2f_b10)
df_oz_concat = pd.concat([df_oz2d_a10,df_oz2d_b10,df_oz2f_a10,df_oz2f_b10])
df_oz_concat.rename(columns={'skaterFullName': 'Player'}, inplace=True)
df_oz_concat['Player'] = df_oz_concat['Player'].str.lower()
display(df_oz_concat.sort_values(by=['OZ%_Rating'], ascending=False))

## PK TOI/GP

In [ ]:
df_pk_init['skaterFullName'] = df_pk_init['skaterFullName'].str.lower()
df_pk = df_pk_init[df_pk_init['skaterFullName'].isin(names_list)]

df_pkf = df_pk.loc[df_pk['positionCode'] != 'D']
df_pkd = df_pk.loc[df_pk['positionCode'] == 'D']

df_pkf_a10 = df_pkf.loc[df_pkf['gamesPlayed'] >= 10]
df_pkf_b10 = df_pkf.loc[df_pkf['gamesPlayed'] < 10]
df_pkd_a10 = df_pkd.loc[df_pkd['gamesPlayed'] >= 10]
df_pkd_b10 = df_pkd.loc[df_pkd['gamesPlayed'] < 10]

df_pkf_a102 = df_pkf_a10.loc[:,['skaterFullName','positionCode','gamesPlayed','shTimeOnIcePerGame']]
df_pkf_a102['pktoi_zscore'] = zscore(df_pkf_a102['shTimeOnIcePerGame'])
z_max_pkf_a102 = df_pkf_a102['pktoi_zscore'].max()
z_min_pkf_a102 = df_pkf_a102['pktoi_zscore'].min()
z_range_pkf_a102 = z_max_pkf_a102-z_min_pkf_a102
z_incr_pkf_a102 = z_range_pkf_a102/rate_range_f_df

df_pkf_b102 = df_pkf_b10.loc[:,['skaterFullName','positionCode','gamesPlayed','shTimeOnIcePerGame']]
df_pkf_b102['pktoi_zscore'] = zscore(df_pkf_b102['shTimeOnIcePerGame'])
z_max_pkf_b102 = df_pkf_b102['pktoi_zscore'].max()
z_min_pkf_b102 = df_pkf_b102['pktoi_zscore'].min()
z_range_pkf_b102 = z_max_pkf_b102-z_min_pkf_b102
z_incr_pkf_b102 = z_range_pkf_b102/(rate_range_f_df-10)

df_pkd_a102 = df_pkd_a10.loc[:,['skaterFullName','positionCode','gamesPlayed','shTimeOnIcePerGame']]
df_pkd_a102['pktoi_zscore'] = zscore(df_pkd_a102['shTimeOnIcePerGame'])
z_max_pkd_a102 = df_pkd_a102['pktoi_zscore'].max()
z_min_pkd_a102 = df_pkd_a102['pktoi_zscore'].min()
z_range_pkd_a102 = z_max_pkd_a102-z_min_pkd_a102
z_incr_pkd_a102 = z_range_pkd_a102/rate_range_d_df

df_pkd_b102 = df_pkd_b10.loc[:,['skaterFullName','positionCode','gamesPlayed','shTimeOnIcePerGame']]
df_pkd_b102['pktoi_zscore'] = zscore(df_pkd_b102['shTimeOnIcePerGame'])
z_max_pkd_b102 = df_pkd_b102['pktoi_zscore'].max()
z_min_pkd_b102 = df_pkd_b102['pktoi_zscore'].min()
z_range_pkd_b102 = z_max_pkd_b102-z_min_pkd_b102
z_incr_pkd_b102 = z_range_pkd_b102/(rate_range_d_df-5)

df_pkf_a102['PKTOI_Rating'] = 90-((z_max_pkf_a102-df_pkf_a102['pktoi_zscore'])/z_incr_pkf_a102)
df_pkf_b102['PKTOI_Rating'] = 70-((z_max_pkf_b102-df_pkf_b102['pktoi_zscore'])/z_incr_pkf_b102)
df_pkd_a102['PKTOI_Rating'] = 100-((z_max_pkd_a102-df_pkd_a102['pktoi_zscore'])/z_incr_pkd_a102)
df_pkd_b102['PKTOI_Rating'] = 83-((z_max_pkd_b102-df_pkd_b102['pktoi_zscore'])/z_incr_pkd_b102)
df_pktoi_concat = pd.concat([df_pkf_a102,df_pkf_b102,df_pkd_a102,df_pkd_b102])
df_pktoi_concat.rename(columns={'skaterFullName': 'Player'}, inplace=True)
df_pktoi_concat['Player'] = df_pktoi_concat['Player'].str.lower()
display(df_pktoi_concat.sort_values(by=['PKTOI_Rating'], ascending=False))

In [ ]:
df_df = df_ca_concat.merge(df_oz_concat, how='left',on='Player').merge(df_pktoi_concat, how='left',on='Player')
df_df['xGA60_Rating'] = df_df['xGA60_Rating'].fillna(55)
df_df['OZ%_Rating'] = df_df['OZ%_Rating'].fillna(55)
df_df['PKTOI_Rating'] = df_df['PKTOI_Rating'].fillna(55)

df_df['DF'] = (df_df['xGA60_Rating'] * 0.50) + (df_df['OZ%_Rating'] * 0.30) + (df_df['PKTOI_Rating'] * 0.20)
df_df_rate = df_df.merge(df_gm_mult, how='left', on='Player')
df_df_rate['DF'] = df_df_rate['DF'] + df_df_rate['add']
df_df_rate = df_df_rate[['Player','DF']]

df_df_rate.sort_values(by='DF',ascending=False,inplace=True)
df_df_rate.head(15).reset_index(drop=True)

# PS

1. Season PS% (70%)
2. Career PS% (30%)

In [ ]:
df_ps_init['skaterFullName'] = df_ps_init['skaterFullName'].str.lower()
df_ps = df_ps_init[df_ps_init['skaterFullName'].isin(names_list)]

df_ps_season = df_ps.loc[:,['skaterFullName','shootoutShootingPct']]
df_ps_career = df_ps.loc[:,['skaterFullName','careerShootoutShots','careerShootoutShootingPct']]

df_ps_season.fillna(0,inplace=True)
df_ps_career.fillna(0,inplace=True)

In [ ]:
df_ps_season['so%season_zscore'] = zscore(df_ps_season['shootoutShootingPct'])
z_max_ps_season = df_ps_season['so%season_zscore'].max()
z_min_ps_season = df_ps_season['so%season_zscore'].min()
z_range_ps_season = z_max_ps_season-z_min_ps_season
z_incr_ps_season = z_range_ps_season/rate_range

df_ps_career['so%career_zscore'] = zscore(df_ps_career['careerShootoutShootingPct'])
z_max_ps_career = df_ps_career['so%career_zscore'].max()
z_min_ps_career = df_ps_career['so%career_zscore'].min()
z_range_ps_career = z_max_ps_career-z_min_ps_career
z_incr_ps_career = z_range_ps_career/rate_range

df_ps_season['Season'] = 99-((z_max_ps_season-df_ps_season['so%season_zscore'])/z_incr_ps_season)
df_ps_career['Career'] = 99-((z_max_ps_career-df_ps_career['so%career_zscore'])/z_incr_ps_career)

df_ps_final = df_ps_season.merge(df_ps_career, how='left',on=['skaterFullName'])
df_ps_final['PS'] = (df_ps_final['Season'] * 0.7) + (df_ps_final['Career'] * 0.3)
df_ps_final['PS'] = np.where(df_ps_final['PS'] >= 99, 85, df_ps_final['PS'])

df_ps_rate = df_ps_final[['skaterFullName','PS']]

df_ps_rate['skaterFullName'] = df_ps_rate['skaterFullName'].str.lower()

df_ps_rate.rename(columns={'skaterFullName': 'Player'}, inplace=True)

df_ps_rate.sort_values(by='PS',ascending=False,inplace=True)
display(df_ps_rate.head(15).reset_index(drop=True))


# EX

1. GP: Up to 1500 (100%)

In [ ]:
df_ex_init['Player'] = df_ex_init['Player'].str.lower()
df_ex = df_ex_init[df_ex_init['Player'].isin(names_list)]

df_ex['GP'] = np.where(df_ex['GP'] > 1400, 1400, df_ex['GP'])
df_ex['gp_zscore'] = zscore(df_ex['GP'])
z_max_ex = df_ex['gp_zscore'].max()
z_min_ex = df_ex['gp_zscore'].min()
z_range_ex = z_max_ex-z_min_ex
z_incr_ex = z_range_ex/rate_range

df_ex['EX'] = 99-((z_max_ex-df_ex['gp_zscore'])/z_incr_ex)
df_ex['name'] = df_ex['Player'].str.lower()
df_ex['name'] = df_ex['Player'].str.replace(' ','-')
df_ex['UrlLink'] = 'https://www.nhl.com/player/' + df_ex['name'] + '-' + df_ex['Id'].astype(str)

df_ex_rate = df_ex[['Id','Player','UrlLink','Birth Date','Birth City','Birth State/Province','Birth Country','Height','Weight','Rookie','EX']]

df_ex_rate['Player'] = df_ex_rate['Player'].str.lower()

df_ex_rate.sort_values(by='EX',ascending=False,inplace=True)

df_ex_rate.head(15).reset_index(drop=True)

df_view = df_ex_rate[['Player','EX']]

df_view.head(15).reset_index(drop=True)

# LD

1. Captaincy/Assistant (80%)
2. GP (20%)

In [ ]:
df_ld_init['Player'] = df_ld_init['Player'].str.lower()
df_ld = df_ld_init[df_ld_init['Player'].isin(names_list)]

df_ld2 = df_ld[df_ld.set_index(['Player']).index.isin(df_ex.set_index(['Player']).index)]
df_ld2['score'] = (df_ld2['Captaincy'] * 3) + (df_ld2['Assistant'] * 1)
df_ld2['score'] = np.where(df_ld2['score'] > 35, 35, df_ld2['score'])

df_ld2['captass_zscore'] = zscore(df_ld2['score'])
z_max_ld = df_ld2['captass_zscore'].max()
z_min_ld = df_ld2['captass_zscore'].min()
z_range_ld = z_max_ld-z_min_ld
z_incr_ld = z_range_ld/(rate_range-10)

df_ld2['LD_Rating'] = 99-((z_max_ld-df_ld2['captass_zscore'])/z_incr_ld)
df_ld_merge = df_ex_rate.merge(df_ld2, how='left', on=['Player']).fillna(58)
df_ld_merge['LD'] = (df_ld_merge['LD_Rating'] * 0.9) + (df_ld_merge['EX'] * 0.1)

df_ld_rate = df_ld_merge[['Player','LD']]

df_ld_rate['Player'] = df_ld_rate['Player'].str.lower()

df_ld_rate.sort_values(by='LD',ascending=False,inplace=True)

df_ld_rate.head(15).reset_index(drop=True)

# SK

In [ ]:
df_ent = df_pos_ent.loc[:,['Player','TOI','Carries/60']]
df_ent['Player'] = df_ent['Player'].str.lower()
df_ent['Player'] = df_ent['Player'].str.split().str.join(' ')
df_ent = df_ent[df_ent['Player'].isin(names_list)]

df_ext = df_pos_ex.loc[:,['Player','TOI','Possessions/60']]
df_ext['Player'] = df_ext['Player'].str.lower()
df_ext['Player'] = df_ext['Player'].str.split().str.join(' ')
df_ext = df_ext[df_ext['Player'].isin(names_list)]
df_ext['Possessions/60'] = np.where(df_ext['Possessions/60'] > 11, 11, df_ext['Possessions/60'])
df_ext['Possessions/60'] = np.where(df_ext['Possessions/60'] <= 2 , 2, df_ext['Possessions/60'])

df_ent_a = df_ent.loc[df_ent['TOI'] > 25]
df_ent_b = df_ent.loc[df_ent['TOI'] <= 25]
df_ext_a = df_ext.loc[df_ext['TOI'] > 25]
df_ext_b = df_ext.loc[df_ext['TOI'] <= 25]

df_ent_a

In [ ]:
df_ent_a['Carries/60'] = np.where(df_ent_a['Carries/60'] >= 21, 21, df_ent_a['Carries/60'])
df_ent_a['carries/60_zscore'] = zscore(df_ent_a['Carries/60'])
z_max_ent_a = df_ent_a['carries/60_zscore'].max()
z_min_ent_a = df_ent_a['carries/60_zscore'].min()
z_range_ent_a = z_max_ent_a-z_min_ent_a
z_incr_ent_a = z_range_ent_a/rate_range

df_ent_b['Carries/60'] = np.where(df_ent_b['Carries/60'] >= 18, 18, df_ent_b['Carries/60'])
df_ent_b['carries/60_zscore'] = zscore(df_ent_b['Carries/60'])
z_max_ent_b = df_ent_b['carries/60_zscore'].max()
z_min_ent_b = df_ent_b['carries/60_zscore'].min()
z_range_ent_b = z_max_ent_b-z_min_ent_b
z_incr_ent_b = z_range_ent_b/(rate_range-20)

df_ent_a['SK'] = 99-((z_max_ent_a-df_ent_a['carries/60_zscore'])/z_incr_ent_a)
df_ent_b['SK'] = 79-((z_max_ent_b-df_ent_b['carries/60_zscore'])/z_incr_ent_b)

df_sk_forwards = pd.concat([df_ent_a,df_ent_b])
df_sk_forwards.fillna(60, inplace=True)
display(df_sk_forwards.sort_values(by=['SK'],ascending=False))

In [ ]:
df_ext_a['pos/60_zscore'] = zscore(df_ext_a['Possessions/60'])
z_max_ext_a = df_ext_a['pos/60_zscore'].max()
z_min_ext_a = df_ext_a['pos/60_zscore'].min()
z_range_ext_a = z_max_ext_a-z_min_ext_a
z_incr_ext_a = z_range_ext_a/rate_range

df_ext_b['pos/60_zscore'] = zscore(df_ext_b['Possessions/60'])
z_max_ext_b = df_ext_b['pos/60_zscore'].max()
z_min_ext_b = df_ext_b['pos/60_zscore'].min()
z_range_ext_b = z_max_ext_b-z_min_ext_b
z_incr_ext_b = z_range_ext_b/(rate_range-20)

df_ext_a['SK'] = 99-((z_max_ext_a-df_ext_a['pos/60_zscore'])/z_incr_ext_a)
df_ext_b['SK'] = 79-((z_max_ext_b-df_ext_b['pos/60_zscore'])/z_incr_ext_b)

df_sk_def = pd.concat([df_ext_a,df_ext_b])

display(df_sk_def.sort_values(by=['SK'],ascending=False))

In [ ]:
df_sk_merge = pd.concat([df_sk_forwards,df_sk_def])
df_sk_merge = df_sk_merge.groupby(['Player']).mean().reset_index()
df_sk_rate = df_sk_merge.merge(df_gm_mult, on='Player', how='left')
df_sk_rate['SK'] = df_sk_rate['SK'] + df_sk_rate['add']
df_sk_rate = df_sk_rate[['Player','SK']]

df_sk_rate.sort_values(by='SK',ascending=False,inplace=True)

df_sk_rate.head(30).reset_index(drop=True)

# OV

1. CK
2. FG
3. DI
4. SK
5. ST
6. EN
7. DU
8. PH
9. FO
10. PA
11. SC
12. DF
13. PS
14. EX
15. LD

In [ ]:
df_ov = df_ck_rate.merge(df_fg_rate, how='left',on=['Player']).merge(df_di_rate, how='left',on=['Player']).merge(df_sk_rate,how='left',on=['Player']).merge(df_st_rate,how='left',on=['Player']).merge(df_en_rate,how='left',on=['Player']).merge(df_du_rate,how='left',on=['Player']).merge(df_ph_rate,how='left',on=['Player']).merge(df_fo_rate,how='left',on=['Player']).merge(df_pa_rate,how='left',on=['Player']).merge(df_sc_rate,how='left',on=['Player']).merge(df_df_rate,how='left',on=['Player']).merge(df_ps_rate,how='left',on=['Player']).merge(df_ex_rate,how='left',on=['Player']).merge(df_ld_rate,how='left',on=['Player'])
df_ov['Rookie'] = np.where(df_ov['Rookie'].astype(str).str.contains('True'), 1, 0)
df_ov['PO'] = 1
df_ov['Contract'] = 1
df_ov['Salary'] = 1
df_ov['Year'] = df_ov['Birth Date'].str.slice(start=0,stop=4)
df_ov['Month'] = df_ov['Birth Date'].str.slice(start=5,stop=7)
df_ov['Day'] = df_ov['Birth Date'].str.slice(start=8)

df_ov = df_ov[['Id','Player','Position','Birth Country','Rookie','Year','Month','Day','Weight','height_in','Contract','Salary','UrlLink','CK','FG','DI','SK','ST','EN','DU','PH','FO','PA','SC','DF','PS','EX','LD','PO']]

df_ov['FG'] = df_ov['FG'].fillna(40)
df_ov.fillna(60,inplace=True)

df_ov_f = df_ov[df_ov['Position'] != 'D']
df_ov_d = df_ov[df_ov['Position'] == 'D']

df_ov_d['OV'] = (df_ov_d['CK'] *0.08) + (df_ov_d['DI'] *0.02) + (df_ov_d['SK'] *0.10) + (df_ov_d['ST'] *0.05) + (df_ov_d['EN'] *0.09) + (df_ov_d['DU'] *0.02) + (df_ov_d['PH'] *0.14) + (df_ov_d['PA'] *0.12) + (df_ov_d['SC'] *0.16) + (df_ov_d['DF'] *0.15) + (df_ov_d['EX'] *0.03) + (df_ov_d['LD'] *0.04)
df_ov_f['OV'] = (df_ov_f['CK'] *0.07) + (df_ov_f['DI'] *0.02) + (df_ov_f['SK'] *0.10) + (df_ov_f['ST'] *0.05) + (df_ov_f['EN'] *0.09) + (df_ov_f['DU'] *0.02) + (df_ov_f['PH'] *0.14) + (df_ov_f['PA'] *0.14) + (df_ov_f['SC'] *0.19) + (df_ov_f['DF'] *0.11) + (df_ov_f['EX'] *0.03) + (df_ov_f['LD'] *0.03) + (df_ov_f['FO'] *0.01)

df_ov_rate = pd.concat([df_ov_d,df_ov_f])
df_ov_rate['Player'] = df_ov_rate['Player'].str.title()
df_ov_rate.drop_duplicates(subset='Player',inplace=True)
df_ov_rate.rename(columns={'height_in': 'Height'}, inplace=True)
df_ov_rate['League'] = 'NHL'

df_to_print = df_ov_rate.sort_values(by=['OV'],ascending=False)

df_to_print.to_csv("NHL_Skater_Ratings_2021-2022.csv",index=False,encoding='utf-8')

df_to_print